In [13]:
import tensorflow as tf

In [14]:
import logging
import time
import sys
str_t = time.strftime('%a-%d-%b-%Y', time.localtime(time.time()))
logging.basicConfig(filename = '{}.log'.format(str_t), format='[%(asctime)s] %(levelname)s %(processName)s %(filename)s - %(message)s', filemode='w')

logger = logging.getLogger()

logger.setLevel(logging.DEBUG)
logger.debug('Logger initiated.')




In [48]:
def create_model(input_shape, **kwargs):
    """
    Create a convolutional deep learning model for classification of 4 type of tumor

    parameters
    ----------------------------------------------------------------------------------------------------------
    input_shape : tuple
        shape of input tensor which we going to feed in our CNN model
    activation : string
        activation function used for every layers int the model
    summary : boolean
        print summary of model articheture
    compile : boolean
        compile the model if arg is true
    """

    params = {
        'input_shape' : input_shape,
        'activation' : 'relu',
        'summary': True,
        'compile': True
    }
    for key, value in kwargs.items():
        params[key] = value
        
    
    try :
        from tensorflow.keras import models, layers, optimizers
        logger.info('models and layers successfully imported from tensorflow')
    except:
        logger.error('Error while importing libraries from tensorflow')
        print('Error while importing libraries from tensorflow')

    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3),strides = (1, 1), activation = params['activation'], input_shape = input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation = params['activation']))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation = params['activation']))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation = params['activation']))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(4, activation = 'sigmoid'))

    if params['compile']:
        model.compile(loss = 'CategoricalCrossentropy', optimizer = optimizers.RMSprop(learning_rate = 1e-4), metrics = ['accuracy'])
    
    if params['summary'] :
        model.summary()

    return model
    

In [49]:
model = create_model(input_shape = (128, 128, 1), summary = True)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 63, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 30, 30, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_25 (Conv2D)          (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_23 (MaxPooli  (None, 14, 14, 128)      

In [50]:
import os

dataset_dir = 'dataset'
train_dir = os.path.join(dataset_dir, 'Training')
test_dir = os.path.join(dataset_dir, 'Testing')
val_dir = os.path.join(dataset_dir, 'validation')

In [51]:
try :
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
except:
    logger.error('Error while import ImageDataGenerator from tensorflow')

train_dataGen = ImageDataGenerator(rescale = 1./255, rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True) 
test_dataGen = ImageDataGenerator(rescale = 1./255,  rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
val_dataGen = ImageDataGenerator(rescale = 1./255, rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [52]:
train_gen = train_dataGen.flow_from_directory(train_dir,color_mode = 'rgb',  batch_size = 16, target_size = (128, 128), class_mode = "categorical")
test_gen = test_dataGen.flow_from_directory(test_dir,color_mode = 'rgb',  batch_size = 16, target_size = (128, 128), class_mode = "categorical")
val_gen = val_dataGen.flow_from_directory(val_dir,color_mode = 'rgb',  batch_size = 16, target_size = (128, 128), class_mode = "categorical")

Found 1200 images belonging to 4 classes.
Found 394 images belonging to 4 classes.
Found 573 images belonging to 4 classes.


In [ ]:
history = model.fit(train_gen, steps_per_epoch = 1200//16, epochs = 1000, validation_data = val_gen, validation_steps = 573//16)


Epoch 1/100


2024-03-24 18:43:23.778497: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_fused_impl.h:625 : INTERNAL: Cudnn graph failed to build: UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(4665): 'conv_op' CUDNN_BACKEND_OPERATION: cudnnFinalize Failed


InternalError: Graph execution error:

Detected at node sequential_8/conv2d_23/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/rajnish8292/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/rajnish8292/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/rajnish8292/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/rajnish8292/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/rajnish8292/.local/lib/python3.10/site-packages/ipyker

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


epoch = range(1, len(acc)+1)

plt.plot(epoch, acc, 'r-', label = 'Traning accuracy')
plt.plot(epoch, val_acc, 'b-', label = 'Validation accuracy')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epoch, loss, 'bo', label = 'Traning loss')
plt.plot(epoch, val_loss, 'b-', label = 'Validation loss')
plt.legend()
plt.figure()

image = tf.keras.image.load_image('D:\workspace\tensorflow\deep_learning\brain_tumor\dataset\Training\meningioma_tumor\mm (1).jpg')